In [93]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math
import pprint
import time
import numpy as np
import pandas as pd

## Travis County Boundary API

In [94]:
root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [ ]:
latitudes = []
longitudes = []
for type_tag in root.findall("node"):
    d = type_tag.attrib
    latitudes.append(float(d["lat"]))
    longitudes.append(float(d["lon"]))

## Region Mapping

In [ ]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [ ]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 3200
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [ ]:
latitude_delta = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
longitude_delta

In [ ]:
lat_lon = zip(latitudes, longitudes)

In [ ]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [ ]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [ ]:
map_point_to_region(30.084446, -97.702885)

In [ ]:
get_representative(13)

In [ ]:
valid_regions = set()
for lat, lon in zip(latitudes, longitudes):
    valid_regions.add(map_point_to_region(lat, lon))

In [ ]:
valid_regions = list(valid_regions)
valid_regions.sort()

In [ ]:
region_coords = [get_representative(i) for i in range(3200)]
pprint.pprint(region_coords)

## Batch calls of distance matrix

In [ ]:
# Prepare data for api call

locations = region_coords
# locations = region_coords + ems_coords + hospital_coords
durations = [[0 for i in range(3200)] for j in range(3200)]

#pprint.pprint(locations)
print(len(locations))

In [91]:
batch_size = 1
num_batches = math.ceil(len(valid_regions) / batch_size)
num_batches_per_round = math.ceil(num_batches/4)
current_branch = 3
remainder = len(valid_regions) % batch_size
print(num_batches)

1948


In [92]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
#     'Authorization': '5b3ce3597851110001cf62485eef3a0ab19d4b3e8ad2d8c88dca3d19',
    'Content-Type': 'application/json; charset=utf-8'
}

In [87]:
for i in range(num_batches_per_round*current_branch, min(num_batches_per_round*current_branch+num_batches_per_round, len(valid_regions))):
    print("batch ", i)
    start = i * batch_size
    sources = list(range(start, start + batch_size))

    # handle last partial batch
    if(sources[batch_size - 1] >= len(locations)):
        sources = sources[:remainder]

    locations = [get_representative(valid_regions[j]) for j in range(len(valid_regions))]

    body = {"locations": locations, "sources": sources}
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

    duration_batch = call.json()['durations']

    for j in range(len(duration_batch)):
        for k in range(len(duration_batch[0])):
            durations[valid_regions[i*batch_size + j]][valid_regions[k]] = duration_batch[j][k]
    time.sleep(0.1)


batch  1461
batch  1462
batch  1463
batch  1464
batch  1465
batch  1466
batch  1467
batch  1468
batch  1469
batch  1470
batch  1471
batch  1472
batch  1473


KeyError: 'durations'

In [43]:
len(valid_regions)/490

3.975510204081633

In [30]:
print(len(durations), len(durations[0]))

467 467


In [78]:
np.save("durations4.npy", durations)

In [ ]:
durations = np.load("durations3.npy", allow_pickle=True)

In [ ]:
import json

d = {"latitude_min": min_lat, "longitude_min": min_lon, "latitude_step": latitude_delta, "longitude_step": longitude_delta}
d["time_matrix"] = durations
with open("grid_info.json", "w") as j:
    json.dump(d, j)